<a href="https://colab.research.google.com/github/Rashmikr1203/project_try/blob/main/Emotion_detection_using_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Performance Comparison: The ensemble method combining various pre-trained models and classifiers resulted in the highest accuracy. The table below summarizes the accuracy of each model and combination:


---


1. MobileNet - 65%
2. EfficientNet - 66%
3. VGG16  - 53%
4. InceptionV3 - 67%
5. ResNet50 - 62%
6. Ensemble - 19%
7. MobileNet + Random Forest - 26%
8. EfficientNet + Random Forest - 26%
9. InceptionV3 + Random Forest - 26%
10. MobileNet + Linear SVC - 19%
11. MobileNet + LightLGBM - 22%
12. EfficientNet + CatBoost  - 24%
13. EfficientNet + XGBoost  - 22%
14. InceptionV3 + XGBoost  - 20%
15. Mobilenet + HistGradientBoost - 26%
16. EfficientNet + CNN - 23%
17. Only Random Forest - crashed multiple times



#Ensemble method


Using the stacking method gave an accuracy of 16%.
Models used in this ensemble
1. MOBILENET
2. EFFICIENTNET
3. VGG16
4. INCEPTIONV3
5. resnet50

##EfficientNet
Accuracy of 66%

In [ ]:
#efficientnet
#2. efficientNet
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained EfficientNetB0 base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = EfficientNetB0(include_top=False, input_shape=input_size)

    # Function to rename layers
    def rename_layers(model, suffix):
        def apply_suffix(layer):
            layer.name = layer.name + suffix
            return layer
        new_model = clone_model(model, clone_function=apply_suffix)
        new_model.set_weights(model.get_weights())
        return new_model

    # Rename layers of base_model to end with "2"
    base_model = rename_layers(base_model, '2')

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax', name='output_layer2')(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
model = create_model(num_classes=num_classes)
history = model.fit(train_generator, epochs=25, validation_data=validation_generator)

model.save('efficientnet_model.h5')

# Step 2: Evaluate the model on the validation set and print accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of EfficientNet model on validation data: {accuracy:.2f}")

model.summary()


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


901/901 ━━━━━━━━━━━━━━━━━━━━ 295s 220ms/step - accuracy: 0.5003 - loss: 1.3053 - val_accuracy: 0.5933 - val_loss: 1.1144
Epoch 2/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 119s 131ms/step - accuracy: 0.6418 - loss: 0.9525 - val_accuracy: 0.6404 - val_loss: 0.9907
Epoch 3/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 118s 131ms/step - accuracy: 0.6923 - loss: 0.8307 - val_accuracy: 0.6377 - val_loss: 1.0012
Epoch 4/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 118s 130ms/step - accuracy: 0.7331 - loss: 0.7188 - val_accuracy: 0.6486 - val_loss: 0.9636
Epoch 5/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 119s 131ms/step - accuracy: 0.7754 - loss: 0.6145 - val_accuracy: 0.6431 - val_loss: 1.0692
Epoch 6/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 119s 132ms/step - accuracy: 0.8173 - loss: 0.5007 - val_accuracy: 0.6616 - val_loss: 1.0207
Epoch 7/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 118s 131ms/step - accuracy: 0.8501 - loss: 0.4097 - val_accuracy: 0.6424 - val_loss: 1.1765
Epoch 8/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 119s 131ms/step - accuracy: 0.8840 - loss: 0.32

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ keras_tensor_88     │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling2          │ (None, 224, 224,  │          0 │ keras_tensor_88[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization2      │ (None, 224, 224,  │          7 │ rescaling2[1][0]  │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_12        │ (None, 224, 224,  │          0 │ normalization2[1… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad2      │ (None, 225, 225,  │          0 │ rescaling_12[1][… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv2 (Conv2D) │ (None, 112, 112,  │        864 │ stem_conv_pad2[1… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn2            │ (None, 112, 112,  │        128 │ stem_conv2[1][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation2    │ (None, 112, 112,  │          0 │ stem_bn2[1][0]    │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv2     │ (None, 112, 112,  │        288 │ stem_activation2… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn2         │ (None, 112, 112,  │        128 │ block1a_dwconv2[… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation2 │ (None, 112, 112,  │          0 │ block1a_bn2[1][0] │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze2 │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape2 │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce2  │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand2  │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite2  │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 12,091,570 (46.13 MB)

 Trainable params: 4,016,515 (15.32 MB)

 Non-trainable params: 42,023 (164.16 KB)

 Optimizer params: 8,033,032 (30.64 MB)

##VGG16
Accuracy of 53%

In [ ]:
#vgg19 with transfer learning
#3. corrected vgg19
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained VGG19 base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = VGG19(include_top=False, input_shape=input_size)

    # Function to rename layers
    def rename_layers(model, suffix):
        def apply_suffix(layer):
            layer.name = layer.name + suffix
            return layer
        new_model = clone_model(model, clone_function=apply_suffix)
        new_model.set_weights(model.get_weights())
        return new_model

    # Rename layers of base_model to end with "3"
    base_model = rename_layers(base_model, '3')

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x.name = 'batch_norm_3'
    outputs = Dense(num_classes, activation='softmax', name='output_layer3')(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
model = create_model(num_classes=num_classes)

# Callbacks for learning rate adjustment and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping]
)

model.save('vgg19_model.h5')

# Step 2: Evaluate the model on the validation set and print accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of VGG19 model on validation data: {accuracy:.2f}")

model.summary()


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Epoch 1/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 237s 253ms/step - accuracy: 0.1868 - loss: nan - val_accuracy: 0.1128 - val_loss: nan - learning_rate: 1.0000e-04
Epoch 2/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 223s 247ms/step - accuracy: 0.1097 - loss: nan - val_accuracy: 0.1128 - val_loss: nan - learning_rate: 1.0000e-04
Epoch 3/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 223s 247ms/step - accuracy: 0.1100 - loss: nan - val_accuracy: 0.1128 - val_loss: nan - learning_rate: 1.0000e-04
Epoch 4/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 223s 247ms/step - accuracy: 0.1103 - loss: nan - val_accuracy: 0.1128 - val_loss: nan - learning_rate: 1.0000e-04
Epoch 5/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 223s 247ms/step - accuracy: 0.1098 - loss: nan - val_accuracy: 0.1128 - val_loss: nan - learning_rate: 1.0000e-04
Epoch 6/25
901/901 ━━━━━━━━━━━━━━━━━━━━ 223s 247ms/step - accuracy: 0.1114 - loss: nan - val_accuracy: 0.1128 - val_loss: nan - learning_rate: 2.

##EfficientNet
Accuracy of 66%

In [ ]:
#inceptionV3
#inceptionV3
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow as tf

# Enable mixed precision training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=16
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=16
)

# Function to rename layers
def rename_layers(model, suffix):
    def apply_suffix(layer):
        layer.name = layer.name + suffix
        return layer
    new_model = clone_model(model, clone_function=apply_suffix)
    new_model.set_weights(model.get_weights())
    return new_model

# Load the InceptionV3 model and rename its layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model = rename_layers(base_model, '4')

# Add custom layers on top of InceptionV3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax', dtype='float32')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for learning rate adjustment, early stopping, and model checkpointing
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_inceptionv3_model.keras', monitor='val_loss', save_best_only=True, mode='min')

# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)

# Evaluate the model on the validation set and print accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of InceptionV3 model on validation data: {accuracy:.2f}")

model.save('inceptionV3_model.h5')

model.summary()

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/25
   2/1802 ━━━━━━━━━━━━━━━━━━━━ 3:11 106ms/step - accuracy: 0.1094 - loss: 2.2584   

W0000 00:00:1719762546.020395     118 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1802/1802 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.4404 - loss: 1.4525

W0000 00:00:1719762808.906880     115 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1802/1802 ━━━━━━━━━━━━━━━━━━━━ 408s 162ms/step - accuracy: 0.4404 - loss: 1.4524 - val_accuracy: 0.6156 - val_loss: 1.0432 - learning_rate: 1.0000e-04
Epoch 2/25
1802/1802 ━━━━━━━━━━━━━━━━━━━━ 185s 103ms/step - accuracy: 0.6444 - loss: 0.9610 - val_accuracy: 0.6398 - val_loss: 0.9772 - learning_rate: 1.0000e-04
Epoch 3/25
1802/1802 ━━━━━━━━━━━━━━━━━━━━ 185s 102ms/step - accuracy: 0.7119 - loss: 0.7866 - val_accuracy: 0.6691 - val_loss: 0.9302 - learning_rate: 1.0000e-04
Epoch 4/25
1802/1802 ━━━━━━━━━━━━━━━━━━━━ 182s 101ms/step - accuracy: 0.7836 - loss: 0.6012 - val_accuracy: 0.6548 - val_loss: 1.0013 - learning_rate: 1.0000e-04
Epoch 5/25
1802/1802 ━━━━━━━━━━━━━━━━━━━━ 182s 101ms/step - accuracy: 0.8412 - loss: 0.4490 - val_accuracy: 0.6623 - val_loss: 1.0947 - learning_rate: 1.0000e-04
Epoch 6/25
1802/1802 ━━━━━━━━━━━━━━━━━━━━ 182s 101ms/step - accuracy: 0.8905 - loss: 0.3146 - val_accuracy: 0.6731 - val_loss: 1.2290 - learning_rate: 1.0000e-04
Epoch 7/25
1802/1802 ━━━━━━━━━━━━━━━━━━

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ keras_tensor_565    │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d4 (Conv2D)    │ (None, 111, 111,  │        864 │ keras_tensor_565… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d4[1][0]     │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation4         │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 109, 109,  │      9,216 │ activation4[1][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_14[1][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_14       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 109, 109,  │     18,432 │ activation_14[1]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_24[1][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_24       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d4      │ (None, 54, 54,    │          0 │ activation_24[1]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, 54, 54,    │      5,120 │ max_pooling2d4[1… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_34[1][0]   │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_34       │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_44 (Conv2D)  │ (None, 52, 52,    │    138,240 │ activation_34[1]… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_44[1][0]   │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_44       │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 71,655,547 (273.34 MB)

 Trainable params: 23,873,703 (91.07 MB)

 Non-trainable params: 34,432 (134.50 KB)

 Optimizer params: 47,747,412 (182.14 MB)

##ResNet50
Accuracy of 62%

In [ ]:
#resnet50
#5. resnet50
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow as tf

# Enable mixed precision training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=16
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=16
)

# Function to rename layers
def rename_layers(model, suffix):
    def apply_suffix(layer):
        layer.name = layer.name + suffix
        return layer
    new_model = clone_model(model, clone_function=apply_suffix)
    new_model.set_weights(model.get_weights())
    return new_model

# Load the ResNet50 model and rename its layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model = rename_layers(base_model, '5')

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax', dtype='float32')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for learning rate adjustment, early stopping, and model checkpointing
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_resnet50_model.keras', monitor='val_loss', save_best_only=True, mode='min')

# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)

# Evaluate the model on the validation set and print accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of ResNet50 model on validation data: {accuracy:.2f}")

model.save('resnet50_model.h5')

model.summary()

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


   2/1802 ━━━━━━━━━━━━━━━━━━━━ 2:52 96ms/step - accuracy: 0.1562 - loss: 2.3800   

W0000 00:00:1719765196.872992     115 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1802/1802 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.4602 - loss: 1.4355

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ keras_tensor_1190   │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad5          │ (None, 230, 230,  │          0 │ keras_tensor_119… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv5         │ (None, 112, 112,  │      9,472 │ conv1_pad5[1][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn5           │ (None, 112, 112,  │        256 │ conv1_conv5[1][0] │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu5         │ (None, 112, 112,  │          0 │ conv1_bn5[1][0]   │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad5          │ (None, 114, 114,  │          0 │ conv1_relu5[1][0] │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool5         │ (None, 56, 56,    │          0 │ pool1_pad5[1][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_con… │ (None, 56, 56,    │      4,160 │ pool1_pool5[1][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn5  │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_rel… │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_con… │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn5  │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_rel… │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_con… │ (None, 56, 56,    │     16,640 │ pool1_pool5[1][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_con… │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn5  │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn5  │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 76,972,955 (293.63 MB)

 Trainable params: 25,639,943 (97.81 MB)

 Non-trainable params: 53,120 (207.50 KB)

 Optimizer params: 51,279,892 (195.62 MB)

##Ensemble
Stacking method. Got an accuracy of 16%

In [ ]:
#ensemble
#final ENSEMBLE
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Average
import numpy as np

# Load the pre-trained models
mobilenet_model = load_model('mobilenet_model.h5')
efficientnet_model = load_model('efficientnet_model.h5')
vgg19_model = load_model('vgg19_model.h5')
inceptionv3_model = load_model('inceptionV3_model.h5')
resnet50_model = load_model('resnet50_model.h5')

# Extract the outputs of each model
mobilenet_output = mobilenet_model.output
efficientnet_output = efficientnet_model.output
vgg19_output = vgg19_model.output
inceptionv3_output = inceptionv3_model.output
resnet50_output = resnet50_model.output

# Combine the outputs using an average layer
combined_output = Average()([mobilenet_output, efficientnet_output, vgg19_output, inceptionv3_output, resnet50_output])

# Create the ensemble model
ensemble_model = Model(inputs=[mobilenet_model.input, efficientnet_model.input, vgg19_model.input, inceptionv3_model.input, resnet50_model.input], outputs=combined_output)

# Compile the ensemble model
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Function to generate predictions from the ensemble model
def predict_ensemble(ensemble_model, data_generator, models):
    # Collect predictions from each model
    predictions = [model.predict(data_generator, verbose=1) for model in models]

    # Convert list of predictions to array
    predictions_array = np.array(predictions)

    # Average the predictions
    average_predictions = np.mean(predictions_array, axis=0)

    return average_predictions

# Evaluate the ensemble model on the validation set
models = [mobilenet_model, efficientnet_model, vgg19_model, inceptionv3_model, resnet50_model]
validation_predictions = predict_ensemble(ensemble_model, validation_generator, models)

# Calculate the accuracy on the validation set
validation_labels = validation_generator.classes
predicted_classes = np.argmax(validation_predictions, axis=1)
accuracy = np.mean(predicted_classes == validation_labels)
print(f"Accuracy of the ensemble model on validation data: {accuracy:.2f}")

ensemble_model.summary()

221/221 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step
221/221 ━━━━━━━━━━━━━━━━━━━━ 20s 68ms/step
221/221 ━━━━━━━━━━━━━━━━━━━━ 18s 76ms/step
221/221 ━━━━━━━━━━━━━━━━━━━━ 37s 105ms/step
221/221 ━━━━━━━━━━━━━━━━━━━━ 23s 74ms/step
Accuracy of the ensemble model on validation data: 0.16


Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ keras_tensor_88     │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling2          │ (None, 224, 224,  │          0 │ keras_tensor_88[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization2      │ (None, 224, 224,  │          7 │ rescaling2[0][0]  │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_12        │ (None, 224, 224,  │          0 │ normalization2[0… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad2      │ (None, 225, 225,  │          0 │ rescaling_12[0][… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv2 (Conv2D) │ (None, 112, 112,  │        864 │ stem_conv_pad2[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn2            │ (None, 112, 112,  │        128 │ stem_conv2[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation2    │ (None, 112, 112,  │          0 │ stem_bn2[0][0]    │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv2     │ (None, 112, 112,  │        288 │ stem_activation2… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn2         │ (None, 112, 112,  │        128 │ block1a_dwconv2[… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation2 │ (None, 112, 112,  │          0 │ block1a_bn2[0][0] │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze2 │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape2 │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce2  │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand2  │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite2  │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 76,925,798 (293.45 MB)

 Trainable params: 76,773,311 (292.87 MB)

 Non-trainable params: 152,487 (595.66 KB)

#MobileNet with
1. radom forest
2. linearSVC
3. histGradientBooster
4. lightGBM


##MobileNet with random forest

Accuracy is 26.
We have extracted the features from the output layer but is recommended to extraxt them from the feature layer as the model we have used is a CNN and is further eplained elsewhere

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained MobileNet base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = MobileNet(include_top=False, input_shape=input_size)

    # Rename the layers
    for layer in base_model.layers:
        layer.name = layer.name + '1'

    # Ensure that the input layer is renamed correctly
    base_model.layers[0].name = "input_1"

    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # use some
    outputs = Dense(num_classes, activation='softmax', name='output_layer1')(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
model = create_model(num_classes=num_classes)
history = model.fit(train_generator, epochs=25, validation_data=validation_generator)

model.save('mobilenet_model.h5')

# Step 2: Evaluate the model on the validation set and print accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of MobileNet model on validation data: {accuracy:.2f}")

model.summary()

# Step 3: Feature extraction using the trained model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('output_layer1').output)

# Extract features
train_features = feature_extractor.predict(train_generator, steps=len(train_generator))
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator))

# Convert features to dataframe
train_features_df = pd.DataFrame(train_features)
validation_features_df = pd.DataFrame(validation_features)

# Print first 10 lines of the features dataframe
print("First 10 lines of the training features dataframe:")
print(train_features_df.head(10))

# Get labels
train_labels = train_generator.classes
validation_labels = validation_generator.classes

# Step 4: Classify emotions using Random Forest Classifier with 100 trees and depth of 10
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10)
rf_model.fit(train_features_df, train_labels)

# Predict on validation set
validation_predictions = rf_model.predict(validation_features_df)

# Step 5: Print the accuracy
accuracy_rf = accuracy_score(validation_labels, validation_predictions)
print(f"Accuracy of the Random Forest model: {accuracy_rf:.2f}")


2024-06-30 14:13:04.007619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 14:13:04.007751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 14:13:04.134889: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  2/901 ━━━━━━━━━━━━━━━━━━━━ 1:13 82ms/step - accuracy: 0.0859 - loss: 2.7492  

I0000 00:00:1719756840.878230     118 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

 Total params: 9,664,343 (36.87 MB)

 Trainable params: 3,214,151 (12.26 MB)

 Non-trainable params: 21,888 (85.50 KB)

 Optimizer params: 6,428,304 (24.52 MB)

901/901 ━━━━━━━━━━━━━━━━━━━━ 47s 51ms/step
221/221 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step
First 10 lines of the training features dataframe:
              0             1             2             3             4  \
0  1.941940e-07  1.140648e-11  1.954015e-07  9.238019e-01  7.607993e-02   
1  5.990914e-03  3.001587e-07  2.180982e-04  1.502113e-05  5.968297e-03   
2  2.471271e-04  1.767866e-10  9.997464e-01  4.933448e-07  4.473954e-10   
3  9.989601e-01  7.879442e-05  9.323981e-04  1.508390e-06  2.193342e-07   
4  9.999918e-01  1.011385e-09  8.071435e-06  1.140131e-07  6.812430e-10   
5  4.355656e-04  7.563886e-07  9.924012e-01  2.305671e-03  5.302557e-05   
6  4.486605e-03  1.714338e-08  1.028560e-06  2.340365e-07  9.955099e-01   
7  8.990807e-01  9.675577e-05  1.988107e-02  3.857839e-07  9.582723e-08   
8  9.829754e-01  7.181155e-09  7.356345e-04  7.353839e-08  1.207312e-02   
9  2.337019e-07  2.871356e-10  5.116409e-08  9.999876e-01  1.176539e-05   

              5             6  
0  1.

## Mobilenet + histGradientBooster

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tqdm import tqdm

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained MobileNet base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = MobileNet(include_top=False, input_shape=input_size)

    # Ensure that the input layer is renamed correctly
    base_model.layers[0].name = "input_1"

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)  # Add Dropout layer with 50% dropout rate
    features = Dense(1024, activation='relu', name='features_layer')(x)  # Add a dense layer for feature extraction
    outputs = Dense(num_classes, activation='softmax', name='output_layer1')(features)
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
print("Training the MobileNet model...")
model = create_model(num_classes=num_classes)
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)
print("MobileNet model training completed.")

model.save('mobilenet_model.h5')

# Step 2: Evaluate the model on the validation set and print accuracy
print("Evaluating the MobileNet model...")
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of MobileNet model on validation data: {accuracy:.2f}")

# Step 3: Feature extraction using the trained model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('features_layer').output)

# Extract features
print("Extracting features from training set...")
train_features = feature_extractor.predict(train_generator, steps=len(train_generator), verbose=1)
print("Training features extraction completed.")

print("Extracting features from validation set...")
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator), verbose=1)
print("Validation features extraction completed.")

# Get labels
train_labels = train_generator.classes
validation_labels = validation_generator.classes

# Step 4: Classify emotions using HistGradientBoostingClassifier with optimized parameters
print("Initializing HistGradientBoostingClassifier...")
hgb_model = HistGradientBoostingClassifier(
    max_iter=200,
    max_depth=10,
    learning_rate=0.1,
    max_bins=255,
    random_state=42
)
print("Training HistGradientBoostingClassifier...")
hgb_model.fit(train_features, train_labels)
print("HistGradientBoostingClassifier training completed.")

# Predict on validation set
print("Predicting on validation set...")
validation_predictions = hgb_model.predict(validation_features)
print("Prediction completed.")

# Step 5: Print the accuracy
accuracy_hgb = accuracy_score(validation_labels, validation_predictions)
print(f"Accuracy of the HistGradientBoostingClassifier model: {accuracy_hgb:.2f}")


KeyboardInterrupt: 

##Mobilenet + LinearSVC

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNet
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import tensorflow as tf

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained MobileNet base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = MobileNet(include_top=False, input_shape=input_size)

    # Ensure that the input layer is renamed correctly
    base_model.layers[0].name = "input_1"

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)  # Add Dropout layer with 50% dropout rate
    features = Dense(1024, activation='relu', name='features_layer')(x)  # Add a dense layer for feature extraction
    outputs = Dense(num_classes, activation='softmax', name='output_layer1')(features)
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
print("Training the MobileNet model...")
model = create_model(num_classes=num_classes)
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)
print("MobileNet model training completed.")

model.save('mobilenet_model.h5')

# Step 2: Evaluate the model on the validation set and print accuracy
print("Evaluating the MobileNet model...")
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of MobileNet model on validation data: {accuracy:.2f}")

# Step 3: Feature extraction using the trained model
print("Extracting features using the trained MobileNet model...")
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('features_layer').output)

# Extract features
train_features = feature_extractor.predict(train_generator, steps=len(train_generator))
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator))
print("Feature extraction completed.")

# Flatten the features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
validation_features_flat = validation_features.reshape(validation_features.shape[0], -1)

# Get labels
train_labels = train_generator.classes
validation_labels = validation_generator.classes

# Step 4: Classify emotions using LinearSVC
print("Training the LinearSVC model...")
linear_svc_model = LinearSVC()

# Fit the model with progress bar
for _ in tqdm(range(1), desc="LinearSVC Training Progress"):
    linear_svc_model.fit(train_features_flat, train_labels)
print("LinearSVC model training completed.")

# Predict on validation set
print("Making predictions with the LinearSVC model...")
validation_predictions = linear_svc_model.predict(validation_features_flat)

# Step 5: Print the accuracy
accuracy_svm = accuracy_score(validation_labels, validation_predictions)
print(f"Accuracy of the LinearSVC model: {accuracy_svm:.2f}")

#model.summary()


##Mobilenet + lightgbm

In [ ]:
#mobilenet and lightLGB
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import tensorflow as tf
import lightgbm as lgb

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained MobileNet base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = MobileNet(include_top=False, input_shape=input_size)

    # Ensure that the input layer is renamed correctly
    base_model.layers[0].name = "input_1"

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)  # Add Dropout layer with 50% dropout rate
    features = Dense(1024, activation='relu', name='features_layer')(x)  # Add a dense layer for feature extraction
    outputs = Dense(num_classes, activation='softmax', name='output_layer1')(features)
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
print("Training the MobileNet model...")
model = create_model(num_classes=num_classes)
history = model.fit(train_generator, epochs=1, validation_data=validation_generator)
print("MobileNet model training completed.")

model.save('mobilenet_model.h5')

# Step 2: Evaluate the model on the validation set and print accuracy
print("Evaluating the MobileNet model...")
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of MobileNet model on validation data: {accuracy:.2f}")

#model.summary()

# Step 3: Feature extraction using the trained model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('features_layer').output)

# Extract features
train_features = feature_extractor.predict(train_generator, steps=len(train_generator))
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator))

# Convert features to dataframe
train_features_df = pd.DataFrame(train_features)
validation_features_df = pd.DataFrame(validation_features)

# Print first 10 lines of the features dataframe
print("First 10 lines of the training features dataframe:")
print(train_features_df.head(10))

# Get labels
train_labels = train_generator.classes
validation_labels = validation_generator.classes

# Step 4: Classify emotions using LightGBM with optimized parameters
lgb_model = lgb.LGBMClassifier(
    n_estimators=500,  # Increase the number of boosting rounds
    max_depth=8,  # Reduce max depth to avoid overfitting
    learning_rate=0.05,  # Decrease learning rate
    subsample=0.8,
    colsample_bytree=0.8,
    num_leaves=1000,  # Reduce num_leaves to avoid overfitting
    min_child_samples=20,  # Increase min_child_samples
    min_child_weight=0.001,  # Increase min_child_weight
    random_state=42
)
lgb_model.fit(train_features_df, train_labels)

# Predict on validation set
validation_predictions = lgb_model.predict(validation_features_df)

# Step 5: Print the accuracy
accuracy_lgb = accuracy_score(validation_labels, validation_predictions)
print(f"Accuracy of the LightGBM model: {accuracy_lgb:.2f}")


# InceptionV3 with

In [ ]:
# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained InceptionV3 base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_size)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)  # Add Dropout layer with 50% dropout rate
    features = Dense(1024, activation='relu', name='features_layer')(x)  # Add a dense layer for feature extraction
    outputs = Dense(num_classes, activation='softmax', name='output_layer')(features)
    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
model = create_model(num_classes=num_classes)
history = model.fit(train_generator, epochs=1, validation_data=validation_generator)

model.save('inceptionv3_model.h5')

# Step 2: Evaluate the model on the validation set and print accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of InceptionV3 model on validation data: {accuracy:.2f}")

# Step 3: Feature extraction using the trained model
# Extract features from the layer just before the output layer
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('global_average_pooling2d').output)

# Extract features
train_features = feature_extractor.predict(train_generator, steps=len(train_generator))
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator))

# Flatten the features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
validation_features_flat = validation_features.reshape(validation_features.shape[0], -1)

# Get labels
train_labels = train_generator.classes
validation_labels = validation_generator.classes

# Step 4: Classify emotions using XGBoost Classifier with 100 trees and depth of 10
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=num_classes, n_estimators=100, max_depth=10)
xgb_model.fit(train_features_flat, train_labels)

# Predict on validation set
validation_predictions = xgb_model.predict(validation_features_flat)

# Step 5: Print the accuracy
accuracy_xgb = accuracy_score(validation_labels, validation_predictions)
print(f"Accuracy of the XGBoost model: {accuracy_xgb:.2f}")

#  Ran in my account has an accuracy of 20 for xgboost and inception V3 is 34
# for about 30 minutes


In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import EfficientNetB0
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Step 1: Train the base model (EfficientNetB0 in this case)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Feature extraction
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add Dropout layer with 50% dropout rate
features = Dense(1024, activation='relu', name='features_layer')(x)  # Feature extraction layer
output = Dense(7, activation='softmax', name='output_layer')(features)  # Output layer

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 2: Train the model
history = model.fit(train_generator, epochs=1, validation_data=validation_generator)

# Step 3: Feature extraction using the trained base model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('features_layer').output)

# Extract features
train_features = feature_extractor.predict(train_generator, steps=len(train_generator))
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator))

# Step 4: Use RandomForestClassifier for classification
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Train RandomForestClassifier on extracted features
rf_model.fit(train_features, train_generator.classes)

# Step 5: Evaluate RandomForestClassifier on validation set
validation_predictions = rf_model.predict(validation_features)
accuracy_rf = accuracy_score(validation_generator.classes, validation_predictions)

print(f"Accuracy of RandomForestClassifier: {accuracy_rf:.2f}")


## accuracy of 26 for amount 7 minutes

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import EfficientNetB0
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for validation data

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Step 1: Train the base model (EfficientNetB0 in this case)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Feature extraction
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.6)(x)  # Increase dropout rate to 60%
features = Dense(1024, activation='relu', name='features_layer')(x)  # Feature extraction layer
output = Dense(7, activation='softmax', name='output_layer')(features)  # Output layer

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 2: Train the model
history = model.fit(train_generator, epochs=2, validation_data=validation_generator)

#checking the accuracy of the inveotionV3 model
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of InceptionV3 model on validation data: {accuracy:.2f}")

# Step 3: Feature extraction using the trained base model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('features_layer').output)

# Extract features
train_features = feature_extractor.predict(train_generator, steps=len(train_generator))
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator))

# Step 4: Use RandomForestClassifier for classification with tuned parameters
rf_model = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)

# Train RandomForestClassifier on extracted features
rf_model.fit(train_features, train_generator.classes)

# Step 5: Evaluate RandomForestClassifier on validation set
validation_predictions = rf_model.predict(validation_features)
accuracy_rf = accuracy_score(validation_generator.classes, validation_predictions)

print(f"Accuracy of RandomForestClassifier: {accuracy_rf:.2f}")


#EfficientNetB0 with

##Nishita

In [ ]:
#code 2 to improve xgboost
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import EfficientNetB0
from sklearn.metrics import accuracy_score
import tensorflow as tf
import xgboost as xgb

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained EfficientNetB0 base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = EfficientNetB0(include_top=False, input_shape=input_size)

    # Freeze the base model layers
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D(name='features_layer')(x)
    x = Dropout(0.5)(x)  # Adding a Dropout layer with 50% dropout rate
    outputs = Dense(num_classes, activation='softmax', name='output_layer')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
model = create_model(num_classes=num_classes)
history = model.fit(train_generator, epochs=2, validation_data=validation_generator)

model.save('efficientnet_model.h5')

# Step 2: Evaluate the model on the validation set and print accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of EfficientNet model on validation data: {accuracy:.2f}")

# Step 3: Feature extraction using the trained model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('features_layer').output)

# Extract features
train_features = feature_extractor.predict(train_generator, steps=len(train_generator), verbose=1)
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator), verbose=1)

# Flatten the features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
validation_features_flat = validation_features.reshape(validation_features.shape[0], -1)

# Get labels
train_labels = train_generator.classes
validation_labels = validation_generator.classes

# Step 4: Classify emotions using XGBoost
xgb_model = xgb.XGBClassifier(n_estimators=200, max_depth=10, learning_rate=0.1, subsample=0.8, colsample_bytree=0.8, random_state=42, verbosity=1)
xgb_model.fit(train_features_flat, train_labels)

# Predict on validation set
validation_predictions = xgb_model.predict(validation_features_flat)

# Step 5: Print the accuracy
accuracy_xgb = accuracy_score(validation_labels, validation_predictions)
print(f"Accuracy of the XGBoost model: {accuracy_xgb:.2f}")


#Accuracy of the XGBoost model: 0.22
#Accuracy of EfficientNet model on validation data: 0.45

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import EfficientNetB0
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained EfficientNetB0 base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = EfficientNetB0(include_top=False, input_shape=input_size)

    # Freeze the base model layers
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D(name='features_layer')(x)
    x = Dropout(0.5)(x)  # Adding a Dropout layer with 50% dropout rate
    outputs = Dense(num_classes, activation='softmax', name='output_layer')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
model = create_model(num_classes=num_classes)
history = model.fit(train_generator, epochs=2, validation_data=validation_generator)

model.save('efficientnet_model.h5')

# Step 2: Evaluate the model on the validation set and print accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of EfficientNet model on validation data: {accuracy:.2f}")

# Step 3: Feature extraction using the trained model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('features_layer').output)

# Extract features
train_features = feature_extractor.predict(train_generator, steps=len(train_generator), verbose=1)
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator), verbose=1)

# Flatten the features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
validation_features_flat = validation_features.reshape(validation_features.shape[0], -1)

# Get labels
train_labels = train_generator.classes
validation_labels = validation_generator.classes

# Step 4: Classify emotions using Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(train_features_flat, train_labels)

# Predict on validation set
validation_predictions = rf_model.predict(validation_features_flat)

# Step 5: Print the accuracy
accuracy_rf = accuracy_score(validation_labels, validation_predictions)
print(f"Accuracy of the Random Forest model: {accuracy_rf:.2f}")

#Accuracy of the Random Forest model: 0.26
#Accuracy of EfficientNet model on validation data: 0.45

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.applications import EfficientNetB0
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define paths
train_path = "/kaggle/input/face-expression-recognition-dataset/images/train"
validation_path = "/kaggle/input/face-expression-recognition-dataset/images/validation"

# Image data generators
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Create image generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    validation_path,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

# Define the model using a pre-trained EfficientNetB0 base
def create_model(input_size=(224, 224, 3), num_classes=7):
    base_model = EfficientNetB0(include_top=False, input_shape=input_size)

    # Freeze the base model layers
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D(name='features_layer')(x)
    x = Dropout(0.5)(x)  # Adding a Dropout layer with 50% dropout rate
    outputs = Dense(num_classes, activation='softmax', name='output_layer')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Determine number of classes
num_classes = len(train_generator.class_indices)

# Step 1: Train the model
model = create_model(num_classes=num_classes)
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Step 2: Evaluate the model on the validation set and print accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f"Accuracy of EfficientNet model on validation data: {accuracy:.2f}")

# Step 3: Feature extraction using the trained model
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('features_layer').output)

# Extract features
train_features = feature_extractor.predict(train_generator, steps=len(train_generator))
validation_features = feature_extractor.predict(validation_generator, steps=len(validation_generator))

# Flatten the features
train_features_flat = train_features.reshape(train_features.shape[0], -1)
validation_features_flat = validation_features.reshape(validation_features.shape[0], -1)

# Get labels
train_labels = train_generator.classes
validation_labels = validation_generator.classes

# Step 4: Classify emotions using Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(train_features_flat, train_labels)

# Predict on validation set
validation_predictions = rf_model.predict(validation_features_flat)

# Step 5: Print the accuracy
accuracy_rf = accuracy_score(validation_labels, validation_predictions)
print(f"Accuracy of the Random Forest model: {accuracy_rf:.2f}")

#Accuracy of the Random Forest model: 0.26
#Accuracy of EfficientNet model on validation data: 0.47

##Aru